In [1]:
import tensorflow as tf
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from nltk.tokenize.casual import TweetTokenizer
from w266_common import utils, vocabulary
import time
import datetime
import os


/home/saulgrimaldo1/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import csv
tokenizer = TweetTokenizer()
x_data = []
x_contexts = []
labels = []
sentences  = []
contexts = []
with open('dta/merged_data_v2.csv', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            continue
        sentence, context, sarcasm = row
        sentences.append(sentence)
        contexts.append(context)
        x_tokens = utils.canonicalize_words(tokenizer.tokenize(sentence), hashtags = True)
        context_tokens = utils.canonicalize_words(tokenizer.tokenize(context), hashtags = True)
        x_data.append(x_tokens)
        x_contexts.append(context_tokens)
        labels.append(int(sarcasm))


#rng = np.random.RandomState(5)
#rng.shuffle(x_data)  # in-place
#train_split_idx = int(0.7 * len(labels))
#test_split_idx  = int(0.9 * len(labels))

shuffle_indices = np.random.permutation(np.arange(len(labels)))
train_split_idx = int(0.7 * len(labels))
test_split_idx  = int(0.9 * len(labels))

train_indices = shuffle_indices[:train_split_idx]
validation_indices = shuffle_indices[train_split_idx:test_split_idx]
test_indices = shuffle_indices[test_split_idx:]


train_sentences = np.array(x_data)[train_indices]
train_contexts = np.array(x_contexts)[train_indices]
train_labels= np.array(labels)[train_indices] 
validation_sentences = np.array(x_data)[validation_indices]
validation_labels = np.array(labels)[validation_indices]
validation_contexts = train_contexts = np.array(x_contexts)[validation_indices]
test_sentences = np.array(x_data)[test_indices]  
test_contexts = train_contexts = np.array(x_contexts)[test_indices]
test_labels = np.array(labels)[test_indices]  

In [3]:
a = [2]*4
a[2] = 6
a

[2, 2, 6, 2]

In [4]:
def transform_labels(raw_label_set, size):
    label_set = []
    for label in raw_label_set:
        labels = [0] * size
        labels[label] = 1
        label_set.append(labels)
    return np.array(label_set)

expanded_train_labels = transform_labels(train_labels, 2)
expanded_validation_labels = transform_labels(validation_labels,2)
expanded_test_labels = transform_labels(test_labels,2)

In [5]:
a = [1,3,4,5,6,7,8,8,1,5,6,7]
a + ["<PADDING>"]*0

[1, 3, 4, 5, 6, 7, 8, 8, 1, 5, 6, 7]

In [6]:
class PaddingAndTruncating:
    def __init__(self, max_len):
        self.max_len = max_len
    
    def pad_or_truncate(self, sentence):
        sen_len = len(sentence)
        paddings = self.max_len - sen_len
        if paddings >=0:
            return sentence + ["<PADDING>"] * paddings
        return sentence[0:paddings]
        
PadAndTrunc = PaddingAndTruncating(10)
        
        
PadAndTrunc.pad_or_truncate(["the","angry","man","is","angry"])

['the',
 'angry',
 'man',
 'is',
 'angry',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [7]:
from tensorflow.contrib import learn

#max_len = max([len(sent) for sent  in train_sentences])
PadAndTrunc =PaddingAndTruncating(30)
train_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, train_sentences))
train_context_padded = list(map(PadAndTrunc.pad_or_truncate, train_contexts))
validation_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, validation_sentences))
validation_context_padded = list(map(PadAndTrunc.pad_or_truncate, validation_contexts))
test_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, test_sentences))
test_context_padded = list(map(PadAndTrunc.pad_or_truncate, test_contexts))

vocab = vocabulary.Vocabulary(utils.flatten(list(train_sentences_padded) + list(train_context_padded)), 2500)
train_s = np.array(list(map(vocab.words_to_ids, train_sentences_padded)))
train_c = np.array(list(map(vocab.words_to_ids, train_context_padded)))
validation_s = np.array(list(map(vocab.words_to_ids, validation_sentences_padded)))
validation_c = np.array(list(map(vocab.words_to_ids, validation_context_padded)))
test_s = np.array(list(map(vocab.words_to_ids, test_sentences_padded)))
test_c = np.array(list(map(vocab.words_to_ids, test_context_padded)))
train_s

array([[   4,    4,    4, ...,    3,    3,    3],
       [  11,    5,    8, ...,    3,    3,    3],
       [  11,    5,    8, ...,    2,   15,    3],
       ..., 
       [  11,    5,    8, ...,    3,    3,    3],
       [  11,    5,    8, ...,    2,   24, 1368],
       [  11,    5,    8, ...,    3,    3,    3]])

In [8]:
i = 3
("conv-maxpool-%s" % i)

'conv-maxpool-3'

In [9]:
class TextCNN(object):

    def __init__(
      self, sequence_length, num_classes, vocab_size, 
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            self.embedded_chars_expanded1 = tf.expand_dims(self.embedded_chars1, -1)
            self.embedded_chars2 = tf.nn.embedding_lookup(self.W, self.input_x2)
            self.embedded_chars_expanded2 = tf.expand_dims(self.embedded_chars2, -1)

        # Create a convolution + avgpool layer for each filter size
        pooled_outputs = []

        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-avgpool-%s" % i) as scope:
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]

                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
              
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded1,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h1 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh1")
               
                # Maxpooling over the outputs
                pooled = tf.nn.avg_pool(
                    h1,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                
                pooled_outputs.append(pooled)
                #pooled_outputs.append(pooled)
                scope.reuse_variables()
                 
                
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded2,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h2 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh2")
                # Maxpooling over the outputs
                pooled2 = tf.nn.avg_pool(
                    h2,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled2)

        # Combine all the pooled features

        num_filters_total = num_filters * len(filter_sizes) * 2

        self.h_pool = tf.concat(pooled_outputs, 1)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])


        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
        
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
           
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            self.labels = tf.argmax(self.input_y, 1)
            TP = tf.count_nonzero(self.predictions * self.labels)
            TN = tf.count_nonzero((self.predictions - 1) * (self.labels - 1))
            FP = tf.count_nonzero(self.predictions * (self.labels - 1))
            FN = tf.count_nonzero((self.predictions - 1) * self.labels)
            correct_predictions = tf.equal(self.predictions, self.labels)
            self.precision = TP / (TP + FP)
            self.recall = TP / (TP + FN)
            self.f1_score = 2 * self.precision * self.recall / (self.precision + self.recall)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [10]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [11]:
learning_rate = 0.2

tf.flags.DEFINE_float("dev_sample_percentage", 0.5, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 100, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 100, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.25, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size = 100
tf.flags.DEFINE_integer("batch_size", batch_size, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 500, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 20, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================


with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():

        cnn = TextCNN(
            sequence_length=train_s.shape[1],
            num_classes=2,
            vocab_size=2500,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
        
         # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdagradOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch1, x_batch2, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch1, x_batch2, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            #r 
            step, summaries, loss, accuracy, recall, precision, f1 = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy,cnn.recall, cnn.precision, cnn.f1_score],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, rec {:g}, pre {:g}, f1 {:g}".format(time_str, step, loss, accuracy, recall, precision, f1))# recall, precision, f1))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(train_s, train_c,  expanded_train_labels)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch1, x_batch2, y_batch = zip(*batch)
            train_step(x_batch1, x_batch1, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(validation_s, validation_c, expanded_validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
print("\nTest Set:")
dev_step(test_s, test_c, expanded_test_labels, writer=dev_summary_writer)
print("")



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=100
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.5
DROPOUT_KEEP_PROB=0.25
EMBEDDING_DIM=100
EVALUATE_EVERY=20
FILTER_SIZES=1
L2_REG_LAMBDA=0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=500
NUM_FILTERS=100
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/hist is illegal; using conv-avgpool-0/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/sparsity is illegal; using conv-avgpool-0/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad/hist is illegal; using conv-avgpool-0/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/gr

2018-04-16T00:50:35.144075: step 101, loss 0.482973, acc 0.83
2018-04-16T00:50:35.196162: step 102, loss 0.484458, acc 0.77
2018-04-16T00:50:35.248579: step 103, loss 0.564229, acc 0.69
2018-04-16T00:50:35.291047: step 104, loss 0.558755, acc 0.794872
2018-04-16T00:50:35.343269: step 105, loss 0.623146, acc 0.69
2018-04-16T00:50:35.400952: step 106, loss 0.584778, acc 0.73
2018-04-16T00:50:35.453741: step 107, loss 0.448503, acc 0.81
2018-04-16T00:50:35.497291: step 108, loss 0.457528, acc 0.75641
2018-04-16T00:50:35.550409: step 109, loss 0.403306, acc 0.87
2018-04-16T00:50:35.606481: step 110, loss 0.53684, acc 0.75
2018-04-16T00:50:35.657819: step 111, loss 0.487241, acc 0.77
2018-04-16T00:50:35.699986: step 112, loss 0.518402, acc 0.730769
2018-04-16T00:50:35.754265: step 113, loss 0.587044, acc 0.75
2018-04-16T00:50:35.806130: step 114, loss 0.484983, acc 0.8
2018-04-16T00:50:35.863528: step 115, loss 0.367488, acc 0.89
2018-04-16T00:50:35.908866: step 116, loss 0.541683, acc 0.80

2018-04-16T00:50:42.092997: step 222, loss 0.428279, acc 0.82
2018-04-16T00:50:42.145114: step 223, loss 0.417949, acc 0.85
2018-04-16T00:50:42.187861: step 224, loss 0.329757, acc 0.833333
2018-04-16T00:50:42.241702: step 225, loss 0.359861, acc 0.86
2018-04-16T00:50:42.300588: step 226, loss 0.426634, acc 0.82
2018-04-16T00:50:42.354433: step 227, loss 0.386732, acc 0.8
2018-04-16T00:50:42.398018: step 228, loss 0.475828, acc 0.833333
2018-04-16T00:50:42.454231: step 229, loss 0.476819, acc 0.83
2018-04-16T00:50:42.523294: step 230, loss 0.367058, acc 0.83
2018-04-16T00:50:42.583804: step 231, loss 0.30273, acc 0.87
2018-04-16T00:50:42.642549: step 232, loss 0.425457, acc 0.807692
2018-04-16T00:50:42.694466: step 233, loss 0.349922, acc 0.86
2018-04-16T00:50:42.750228: step 234, loss 0.394163, acc 0.84
2018-04-16T00:50:42.802954: step 235, loss 0.449114, acc 0.83
2018-04-16T00:50:42.847630: step 236, loss 0.476021, acc 0.74359
2018-04-16T00:50:42.901484: step 237, loss 0.465869, acc 

2018-04-16T00:50:48.946166: step 341, loss 0.407904, acc 0.85
2018-04-16T00:50:49.002058: step 342, loss 0.401701, acc 0.83
2018-04-16T00:50:49.055263: step 343, loss 0.31163, acc 0.89
2018-04-16T00:50:49.099032: step 344, loss 0.241884, acc 0.910256
2018-04-16T00:50:49.157654: step 345, loss 0.291629, acc 0.88
2018-04-16T00:50:49.209896: step 346, loss 0.375548, acc 0.86
2018-04-16T00:50:49.261828: step 347, loss 0.29777, acc 0.83
2018-04-16T00:50:49.304968: step 348, loss 0.408635, acc 0.858974
2018-04-16T00:50:49.363832: step 349, loss 0.251233, acc 0.89
2018-04-16T00:50:49.415776: step 350, loss 0.272566, acc 0.92
2018-04-16T00:50:49.467519: step 351, loss 0.269021, acc 0.92
2018-04-16T00:50:49.511158: step 352, loss 0.382392, acc 0.833333
2018-04-16T00:50:49.563558: step 353, loss 0.333392, acc 0.85
2018-04-16T00:50:49.622742: step 354, loss 0.294289, acc 0.89
2018-04-16T00:50:49.675240: step 355, loss 0.306588, acc 0.87
2018-04-16T00:50:49.718201: step 356, loss 0.386538, acc 0.8

2018-04-16T00:50:55.699606: step 460, loss 1.51418, acc 0.493369, rec 0.967742, pre 0.493151, f1 0.653358

2018-04-16T00:50:55.755368: step 461, loss 0.207007, acc 0.93
2018-04-16T00:50:55.808577: step 462, loss 0.268762, acc 0.91
2018-04-16T00:50:55.861088: step 463, loss 0.309165, acc 0.86
2018-04-16T00:50:55.904170: step 464, loss 0.211416, acc 0.897436
2018-04-16T00:50:55.959811: step 465, loss 0.197248, acc 0.93
2018-04-16T00:50:56.012452: step 466, loss 0.218443, acc 0.89
2018-04-16T00:50:56.064686: step 467, loss 0.198687, acc 0.91
2018-04-16T00:50:56.107831: step 468, loss 0.288241, acc 0.858974
2018-04-16T00:50:56.160163: step 469, loss 0.194544, acc 0.92
2018-04-16T00:50:56.215845: step 470, loss 0.267658, acc 0.91
2018-04-16T00:50:56.268395: step 471, loss 0.236043, acc 0.91
2018-04-16T00:50:56.312319: step 472, loss 0.197612, acc 0.910256
2018-04-16T00:50:56.365941: step 473, loss 0.19857, acc 0.96
2018-04-16T00:50:56.423476: step 474, loss 0.268318, acc 0.89
2018-04-16T00:

...

2018-04-16T00:51:52.596710: step 1405, loss 0.0307073, acc 0.99
2018-04-16T00:51:52.650689: step 1406, loss 0.0238998, acc 1
2018-04-16T00:51:52.706476: step 1407, loss 0.0447922, acc 0.98
2018-04-16T00:51:52.750384: step 1408, loss 0.0784983, acc 0.961538
2018-04-16T00:51:52.807191: step 1409, loss 0.0844332, acc 0.97
2018-04-16T00:51:52.866913: step 1410, loss 0.0746431, acc 0.98
2018-04-16T00:51:52.947037: step 1411, loss 0.0487828, acc 0.97
2018-04-16T00:51:53.018925: step 1412, loss 0.0398343, acc 0.987179
2018-04-16T00:51:53.102271: step 1413, loss 0.0278387, acc 0.99
2018-04-16T00:51:53.185138: step 1414, loss 0.025201, acc 0.99
2018-04-16T00:51:53.272761: step 1415, loss 0.0546221, acc 0.99
2018-04-16T00:51:53.341591: step 1416, loss 0.0191541, acc 1
2018-04-16T00:51:53.425274: step 1417, loss 0.0333778, acc 0.99
2018-04-16T00:51:53.515421: step 1418, loss 0.0452002, acc 0.97
2018-04-16T00:51:53.599870: step 1419, loss 0.0302445, acc 0.99
2018-04-16T00:51:53.668194: step 1420, 

2018-04-16T00:52:00.465987: step 1525, loss 0.0135093, acc 1
2018-04-16T00:52:00.520532: step 1526, loss 0.0361836, acc 0.99
2018-04-16T00:52:00.577804: step 1527, loss 0.0112239, acc 1
2018-04-16T00:52:00.624010: step 1528, loss 0.0362256, acc 0.974359
2018-04-16T00:52:00.682644: step 1529, loss 0.00887161, acc 1
2018-04-16T00:52:00.739752: step 1530, loss 0.0245841, acc 0.99
2018-04-16T00:52:00.794031: step 1531, loss 0.0188469, acc 1
2018-04-16T00:52:00.839391: step 1532, loss 0.0141256, acc 1
2018-04-16T00:52:00.904538: step 1533, loss 0.0378216, acc 0.99
2018-04-16T00:52:00.968912: step 1534, loss 0.0141803, acc 1
2018-04-16T00:52:01.022017: step 1535, loss 0.0551792, acc 0.98
2018-04-16T00:52:01.065961: step 1536, loss 0.0948722, acc 0.974359
2018-04-16T00:52:01.126263: step 1537, loss 0.046698, acc 0.98
2018-04-16T00:52:01.183836: step 1538, loss 0.029813, acc 0.99
2018-04-16T00:52:01.236246: step 1539, loss 0.0430203, acc 0.99
2018-04-16T00:52:01.279222: step 1540, loss 0.04350

2018-04-16T00:52:07.618104: step 1645, loss 0.0238601, acc 0.99
2018-04-16T00:52:07.672670: step 1646, loss 0.0141624, acc 1
2018-04-16T00:52:07.763933: step 1647, loss 0.0170964, acc 0.99
2018-04-16T00:52:07.830200: step 1648, loss 0.0219594, acc 1
2018-04-16T00:52:07.887212: step 1649, loss 0.0297371, acc 0.99
2018-04-16T00:52:07.942196: step 1650, loss 0.0167779, acc 1
2018-04-16T00:52:07.996708: step 1651, loss 0.0321615, acc 0.99
2018-04-16T00:52:08.044823: step 1652, loss 0.035186, acc 0.987179
2018-04-16T00:52:08.098075: step 1653, loss 0.0215082, acc 0.99
2018-04-16T00:52:08.151638: step 1654, loss 0.00967932, acc 1
2018-04-16T00:52:08.208848: step 1655, loss 0.0135795, acc 1
2018-04-16T00:52:08.256044: step 1656, loss 0.0246696, acc 0.987179
2018-04-16T00:52:08.308431: step 1657, loss 0.0241728, acc 1
2018-04-16T00:52:08.361113: step 1658, loss 0.0183557, acc 1
2018-04-16T00:52:08.413042: step 1659, loss 0.0363264, acc 0.99
2018-04-16T00:52:08.455599: step 1660, loss 0.0070515

2018-04-16T00:52:14.691341: step 1766, loss 0.0153067, acc 1
2018-04-16T00:52:14.748669: step 1767, loss 0.0163217, acc 1
2018-04-16T00:52:14.792149: step 1768, loss 0.0195627, acc 0.987179
2018-04-16T00:52:14.845009: step 1769, loss 0.0141296, acc 1
2018-04-16T00:52:14.900801: step 1770, loss 0.0115355, acc 1
2018-04-16T00:52:14.953464: step 1771, loss 0.0204043, acc 1
2018-04-16T00:52:14.996389: step 1772, loss 0.0447402, acc 1
2018-04-16T00:52:15.048602: step 1773, loss 0.0270395, acc 0.99
2018-04-16T00:52:15.100289: step 1774, loss 0.0270511, acc 0.99
2018-04-16T00:52:15.158905: step 1775, loss 0.011837, acc 1
2018-04-16T00:52:15.203168: step 1776, loss 0.0150486, acc 1
2018-04-16T00:52:15.260522: step 1777, loss 0.00839266, acc 1
2018-04-16T00:52:15.316024: step 1778, loss 0.0264811, acc 0.98
2018-04-16T00:52:15.372136: step 1779, loss 0.00891403, acc 1
2018-04-16T00:52:15.414533: step 1780, loss 0.0223365, acc 1

Evaluation:
2018-04-16T00:52:15.515550: step 1780, loss 0.83526, ac

2018-04-16T00:52:21.778344: step 1888, loss 0.0294561, acc 0.987179
2018-04-16T00:52:21.831312: step 1889, loss 0.0203735, acc 0.99
2018-04-16T00:52:21.884962: step 1890, loss 0.0142894, acc 1
2018-04-16T00:52:21.938567: step 1891, loss 0.00991929, acc 1
2018-04-16T00:52:21.985534: step 1892, loss 0.00627188, acc 1
2018-04-16T00:52:22.037110: step 1893, loss 0.00662401, acc 1
2018-04-16T00:52:22.088403: step 1894, loss 0.010249, acc 1
2018-04-16T00:52:22.139942: step 1895, loss 0.0160418, acc 0.99
2018-04-16T00:52:22.182233: step 1896, loss 0.0249288, acc 0.987179
2018-04-16T00:52:22.236978: step 1897, loss 0.0479809, acc 0.98
2018-04-16T00:52:22.290033: step 1898, loss 0.0365667, acc 1
2018-04-16T00:52:22.342418: step 1899, loss 0.0142174, acc 1
2018-04-16T00:52:22.385249: step 1900, loss 0.00643265, acc 1

Evaluation:
2018-04-16T00:52:22.489363: step 1900, loss 0.895297, acc 0.734748, rec 0.669355, pre 0.763804, f1 0.713467

Saved model checkpoint to /home/saulgrimaldo1/sarcasm_detec